In [40]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)

df = pd.read_csv("../../our_analyses/dataset_prepared.csv")
from sklearn.preprocessing import LabelEncoder

df=df.drop(['name', 'artists', 'album_name', 'key', 'mode', 'time_signature'], axis=1)

genre_groups = {
    'idm': 0, 'iranian': 0, 'study': 0,  # Electronic/Dance
    'black-metal': 1, 'breakbeat': 1, 'techno': 1,  # Metal/Rock
    'brazil': 2, 'forro': 2, 'happy': 2, 'spanish': 2, 'j-idol': 2,  # Country/Folk/Pop
    'afrobeat': 3, 'chicago-house': 3, 'industrial': 3, 'j-dance': 3,  # World/Commercial Pop
    'bluegrass': 4, 'disney': 4, 'indian': 4, 'mandopop': 4, 'sleep': 4  # Other
}



# Mappatura dei generi del dataframe ai gruppi di generi
df['genre_group'] = df['genre'].map(genre_groups).astype(int)

# Stampa del mapping dei generi ai numeri corrispondenti
print(df['genre_group'].unique())

le = LabelEncoder()
df['explicit'] = le.fit_transform(df['explicit'])

from sklearn.model_selection import train_test_split

attributes = [col for col in df.columns if col != 'genre_group']
# Definizione di y come l'array delle etichette dei generi
y = np.array(df['genre_group'])

# Rimozione delle colonne 'genre_group' e 'genre' per ottenere le feature
X = df.drop(['genre_group', 'genre'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y) # we use 70% and 30% and we stratisfy to have the same distribution for the classes between them in the training and in the test set


from sklearn.preprocessing import StandardScaler

norm = StandardScaler()
norm.fit(X_train)

X_train_norm = norm.transform(X_train)
X_test_norm = norm.transform(X_test)
df.head

[3 1 4 2 0]


<bound method NDFrame.head of        duration_ms  explicit  popularity  danceability  energy  loudness  \
0           293106         0          50         0.401   0.683    -5.722   
1           194972         0          52         0.672   0.858    -5.233   
2           178428         0          22         0.636   0.826    -7.486   
3           238373         0          20         0.733   0.862    -5.813   
4           221893         0          22         0.712   0.225   -10.017   
...            ...       ...         ...           ...     ...       ...   
14995       238160         0          15         0.545   0.601    -7.265   
14996       294666         0          18         0.643   0.676    -9.708   
14997       234005         0          54         0.673   0.840    -7.564   
14998       261093         0          56         0.658   0.878    -6.834   
14999       308466         0          43         0.465   0.742   -12.131   

       speechiness  acousticness  instrumentalness  liven

In [41]:
optimal_params = {
    'criterion': 'entropy', 'max_depth': 9, 'max_features': None, 
    'min_impurity_decrease': 0.001, 'min_samples_leaf': 10, 'min_samples_split': 2, 'splitter': 'best'
}

# Train the Decision Tree Classifier with optimal parameters on normalized data
dt = DecisionTreeClassifier(**optimal_params)
dt.fit(X_train_norm, y_train)  # Use normalized training data

DecisionTreeClassifier(criterion='entropy', max_depth=9,
                       min_impurity_decrease=0.001, min_samples_leaf=10)

In [42]:
# from sklearn.naive_bayes import GaussianNB, CategoricalNB

# dt = GaussianNB()
# dt.fit(X_train_norm, y_train)

# y_train_pred = dt.predict(X_train_norm)

In [43]:
# from sklearn.neighbors import KNeighborsClassifier

# dt = KNeighborsClassifier(n_neighbors=13, metric="cityblock", weights="distance")
# dt.fit(X_train_norm, y_train)

In [44]:


# Carica il nuovo dataset
df_test = pd.read_csv("../../our_analyses/dataset_test_prepared.csv")

# Preprocessamento dei dati (stesse trasformazioni applicate al dataset di training)
# Rimozione delle colonne non necessarie
df_test = df_test.drop(['name', 'artists', 'album_name', 'key', 'mode', 'time_signature'], axis=1)

# Mappatura dei generi ai gruppi di generi
df_test['genre_group'] = df_test['genre'].map(genre_groups).astype(int)

# Codifica della colonna 'explicit'
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_test['explicit'] = le.fit_transform(df_test['explicit'])

# Separazione delle feature e delle etichette
X_test_new = df_test.drop(['genre_group', 'genre'], axis=1)
y_test_new = np.array(df_test['genre_group'])

# Normalizzazione dei dati del test set
norm.fit(X_train)  # Adatta il normalizzatore ai dati di training (già fatto nel tuo codice)
X_test_new_norm = norm.transform(X_test_new)

# Carica il classificatore Decision Tree (già addestrato)
# Assicurati che 'dt' sia il classificatore Decision Tree che hai addestrato precedentemente
# dt = DecisionTreeClassifier(**optimal_params)
# dt.fit(X_train_norm, y_train)  # Già fatto nel tuo codice

# Fai le previsioni sul nuovo dataset
predictions = dt.predict(X_test_new_norm)

# Qui puoi calcolare metriche di valutazione se necessario
print("Accuracy:", accuracy_score(y_test_new, predictions))

Accuracy: 0.6196


In [46]:
import plotly.graph_objects as go
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve, average_precision_score


from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go
# Binarizza le etichette in un formato one-vs-rest e calcola i punteggi di probabilità per le classi
Y = label_binarize(y_test, classes=np.unique(y_test))
y_scores = dt.predict_proba(X_test_norm)

# Calcola ROC curve e AUC per ogni classe
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(np.unique(y_test))):
    fpr[i], tpr[i], _ = roc_curve(Y[:, i], y_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Crea un grafico Plotly
fig = go.Figure()

# Aggiunge una linea per ogni classe
for i, genre in enumerate(np.unique(y_test)):
    fig.add_trace(go.Scatter(x=fpr[i], y=tpr[i], mode='lines', name=f'Class {i} (AUC={roc_auc[i]:0.2f})'))

# Aggiunge la linea diagonale per il random guessing
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Guessing', line=dict(dash='dash')))

# Aggiorna il layout
fig.update_layout(
    title="ROC curves of Decision Tree classifier",
    xaxis_title="False Positive Rate",
     width=1000,  # Larghezza del plot
    height=800,  
    yaxis_title="True Positive Rate",
    legend_title="Classes"
)

# Mostra il grafico
fig.show()
